# Check the skipped instances from MA

In [7]:
data[0]['problem']

'sepsis'

In [2]:
import json
# missing_qn = []
missing_id = []
# Replace 'your_file.json' with the path to your JSON file
with open('/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_acute_kidney_injury_new_temp.json', 'r') as file:
    data = json.load(file)
# print("Total number of items:", len(data))

for idx, item in enumerate(data):
    # print(item['final']['final_choice'])
    if 'final' not in item.keys():
        print("Error in line", idx)
        print(item['hadm_id'])
        missing_id.append(item['adm_id'])
#     try:
#         question = item['Question']
#     except:
#         print("Error in line", idx)
#         print(item['qn_num'])
#         missing_qn.append(item['qn_num'])

In [18]:
# print(missing_qn)
print(missing_id)

[]


# Check the skipped instances from Baseline

In [3]:
import json
missing_item = []
# Replace 'your_file.json' with the path to your JSON file
with open('/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_sepsis_baseline.json', 'r') as file:
    data = json.load(file)

for idx, item in enumerate(data):
    if item["BaselineChoice"] == "Unknown":
        print("Error in line", idx)
        print(item['hadm_id'])
        missing_item.append(item['hadm_id'])

In [4]:
data[0]

{'note': 'pleuritic right chest pain\n- patient started on coumadin\n- [** 949**] briefly decreased (although unclear whether this was due to\naccidental foley removal), [** 949**] responded appropriately to 500 ml lr\nbolus with [** 949**] 60-70 cc\ns per hour this am\n- patient briefly developed non-gap acidosis with bicarb of 19,\nresolved with substitution of lr for ns for fluid boluses\ns: the patient is feeling well.  her chest pain has entirely resolved.\nshe continues to have dry-mouth as well as her baseline amount of cough\nand shortness or breath.  she has not had n/v diarrhea.  no pain or\nswelling in her legs.\nlosartan\nangioedema/lip\nlisinopril (oral)\ncough;\npenicillins\nitching;\nflagyl (oral) (metronidazole)\nwheezing;\nultram (oral) (tramadol hcl)\nrash;\nreview of systems is unchanged from admission except as noted below\nreview of systems:\nlast dose of antibiotics:\nciprofloxacin - [**2141-7-18**] 08:00 pm\nvancomycin - [**2141-7-19**] 08:02 am\ninfusions:\nhepa

In [16]:
import json
import copy
import os

def deep_merge(dict1, dict2):
    """
    Recursively merge dict2 into dict1.
    - If a key exists in both dicts and both values are themselves dicts, 
      recurse into those.
    - Otherwise, dict2’s value overwrites dict1’s value.
    """
    for key, value in dict2.items():
        if key not in dict1:
            dict1[key] = value
        else:
            if isinstance(dict1[key], dict) and isinstance(value, dict):
                deep_merge(dict1[key], value)
            else:
                # Overwrite dict1's value with dict2's value
                dict1[key] = value
    return dict1

def merge_json_files(file1, file2, output_file, identifier='qn_num'):
    """
    Merges two JSON files (each containing a list of dicts) 
    on a specified 'identifier' key. 
    If the key is missing from a particular dict, it will be skipped.
    """
    # 1) Load both JSON files
    with open(file1, "r", encoding="utf-8") as f:
        data1 = json.load(f)  # List of dicts
    with open(file2, "r", encoding="utf-8") as f:
        data2 = json.load(f)  # List of dicts

    # 2) Convert the first list into a dictionary keyed by 'identifier'
    merged_dict = {}
    for item in data1:
        if identifier in item:
            key_value = item[identifier]
            merged_dict[key_value] = copy.deepcopy(item)

    # 3) For each item in data2, either add or merge
    for item in data2:
        if identifier not in item:
            # Skip if 'identifier' doesn't exist in this item
            continue
        key_value = item[identifier]
        if key_value not in merged_dict:
            merged_dict[key_value] = copy.deepcopy(item)
        else:
            # Deep-merge fields from item into existing
            deep_merge(merged_dict[key_value], item)

    # 4) Convert the merged dictionary back to a list
    final_list = list(merged_dict.values())

    print(f"Merged {len(data1)} items from file1 and {len(data2)} items from file2.")
    print(f"Final list contains {len(final_list)} items.")

    # 5) Write to the output file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(final_list, f, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    # Example usage:
    file1 = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_missing_final.json"
    file2 = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json"
    output_file = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json"

    # Example identifier: 'File ID' or 'qn_num'
    # If you need to merge on 'File ID', just pass identifier='File ID'.
    merge_json_files(file1, file2, output_file, identifier='File ID')
    print(f"Merged file saved as: {output_file}")


Merged 1 items from file1 and 351 items from file2.
Final list contains 351 items.
Merged file saved as: /home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json


In [17]:
import pandas as pd
# df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/Input/SOAP_5_problems.csv")
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/Input/step3_ALL.csv")
len(df)

137

In [13]:
json_path = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/sepsis_final.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
len(data)

351

# Evaluation

In [11]:
import json

def evaluate_SOAP_predictions(
    file_path,
    terms_lst: list
):
   
    def get_label(label_str, terms_lst):
        for term in terms_lst:
            if term.lower() in label_str.lower():
                return "Yes"
        return "No"

    def compute_confusion_matrix(data_list, baseline=False):
      
        tp = fp = tn = fn = other = 0

        for item in data_list:
            pred = item["BaselineChoice"] if baseline else item["final"]['final_choice']
            label = get_label(item["label"], terms_lst)

            if pred == "Yes" and label == "Yes":
                tp += 1
            elif pred == "Yes" and label == "No":
                fp += 1
            elif pred == "No" and label == "No":
                tn += 1
            elif pred == "No" and label == "Yes":
                fn += 1
            else:
                # e.g., pred is "Yes" but label is "Not sure", or vice versa
                other += 1

        return tp, tn, fp, fn, other

    def calculate_metrics(tp, tn, fp, fn):
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        recall = tp / (tp + fn) if (tp + fn) else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0.0

        return {
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        }

    def print_results(title, tp, tn, fp, fn, other, metrics):
        """
        Nicely prints out the confusion matrix and performance metrics.
        """
        print(f"\n{title}")
        print("-" * len(title))
        print(f"{'True Positives (TP)':25} : {tp}")
        print(f"{'True Negatives (TN)':25} : {tn}")
        print(f"{'False Positives (FP)':25} : {fp}")
        print(f"{'False Negatives (FN)':25} : {fn}")
        # print(f"{'Other':25} : {other}")

        print()
        for metric_name, value in metrics.items():
            print(f"{metric_name:25} : {value:.3f}")

    # ----------------------------
    # 2. Load data from JSON file
    # ----------------------------
    with open(file_path, "r") as file:
        data = json.load(file)

    # ---------------------------------------------------
    # 3. Compute confusion matrix & metrics for aggregator
    # ---------------------------------------------------
    agg_tp, agg_tn, agg_fp, agg_fn, agg_other = compute_confusion_matrix(data)
    agg_metrics = calculate_metrics(agg_tp, agg_tn, agg_fp, agg_fn)

    # -------------------------------------------------
    # 4. Compute confusion matrix & metrics for baseline
    # -------------------------------------------------
    # base_tp, base_tn, base_fp, base_fn, base_other = compute_confusion_matrix(data, baseline=True)
    # base_metrics = calculate_metrics(base_tp, base_tn, base_fp, base_fn)

    print_results("Multi-Agents", agg_tp, agg_tn, agg_fp, agg_fn, agg_other, agg_metrics)
    # print_results("Baseline (ZS COT)", base_tp, base_tn, base_fp, base_fn, base_other, base_metrics)

In [10]:
mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]

print("Sepsis")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_sepsis_baseline.json", sepsis)

print("AKI")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_acute_kidney_injury_baseline.json", aki)

print("Congestive Heart Failure")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_congestive_heart_failure_baseline.json", chf)

Sepsis

Multi-Agents
------------
True Positives (TP)       : 114
True Negatives (TN)       : 79
False Positives (FP)      : 83
False Negatives (FN)      : 12

Precision                 : 0.579
Recall                    : 0.905
F1 Score                  : 0.706

Baseline (ZS COT)
-----------------
True Positives (TP)       : 105
True Negatives (TN)       : 94
False Positives (FP)      : 68
False Negatives (FN)      : 21

Precision                 : 0.607
Recall                    : 0.833
F1 Score                  : 0.702
AKI

Multi-Agents
------------
True Positives (TP)       : 77
True Negatives (TN)       : 93
False Positives (FP)      : 63
False Negatives (FN)      : 55

Precision                 : 0.550
Recall                    : 0.583
F1 Score                  : 0.566

Baseline (ZS COT)
-----------------
True Positives (TP)       : 82
True Negatives (TN)       : 92
False Positives (FP)      : 64
False Negatives (FN)      : 50

Precision                 : 0.562
Recall             

In [13]:
mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]
print("================New temp================")
print("Sepsis")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_sepsis_new_temp.json", sepsis)
print("AKI")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_acute_kidney_injury_new_temp.json", aki)
print("Congestive Heart Failure")
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/3_problems_congestive_heart_failure_new_temp.json", chf)

================New temp================
Sepsis

Multi-Agents
------------
True Positives (TP)       : 108
True Negatives (TN)       : 80
False Positives (FP)      : 82
False Negatives (FN)      : 18

Precision                 : 0.568
Recall                    : 0.857
F1 Score                  : 0.684
AKI

Multi-Agents
------------
True Positives (TP)       : 79
True Negatives (TN)       : 96
False Positives (FP)      : 60
False Negatives (FN)      : 53

Precision                 : 0.568
Recall                    : 0.598
F1 Score                  : 0.583
Congestive Heart Failure

Multi-Agents
------------
True Positives (TP)       : 36
True Negatives (TN)       : 167
False Positives (FP)      : 29
False Negatives (FN)      : 56

Precision                 : 0.554
Recall                    : 0.391
F1 Score                  : 0.459


In [ ]:
import json
with open("/home/yl3427/cylab/SOAP_MA/Output/SOAP/1_temp_ma5_base5_3problems_auto/3_problems_acute_kidney_injury_baseline.json", "r") as f:
    data = json.load(f)

for i in range(len(data)):
    n=1
    panel_dict = data[i].get(f"panel_{n}", None)
    while panel_dict:
        num_specialists = len(panel_dict['Initially Identified Specialties'])
        print(f"Panel {n} has {num_specialists} specialists.")
        n += 1
        panel_dict = data[i].get(f"panel_{n}", None)

In [1]:
from utils import count_llama_tokens
import json
json_path = "/secure/shared_data/SOAP/MIMIC/cases_base.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for key, dict in data.items():
    result = dict['before_diagnosis'] + dict['after_diagnosis']
    print(count_llama_tokens(result))

/home/yl3427/miniconda3/envs/vllm_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4466
5152
3991
2523
3746
2807
2993
5775
1709
3113
4229
2840
6797
4559
2086
3525
1290
3977
2651
3852
6817
3737
4537
4457
2294
1178
5461
4404
2751
1870
3701
3238
3415
5607
5325
3086
6598
3651
2732
2954
3211
4345
4426
2067
4111
3934
7008
3137
2376
2461
7374
1945
3114
3197
3105
991
3747
5528
3810
4442
2456
4062
3405
5249
5164
3433
6695
4870
7278
4913
4482
2378
6839
7352
3315
1712
6039
4373
4360
3672
9365
4559
4112
5230
2785
5056
3503
3707
2664
10223
2797
2643
2756
1786
2936
4652
928
3450
1829
2878
2610
4383
3678
3688
6681
7336
4353
1728
4607
6216
3110
5023
4715
3566
5617
1645
5237
4919
3651
4161
4432
2616
3267
4349
4454
3558
3760
4408
3425
2657
3171
1949
2123
1683
1751
5554
3073
1353
6019
3086
7439
3864
2616
3707
5054
1206
3234
4055
822
3409
2316
8598
4100
4444
4762
3106
5502
2388
2786
4682
5825
6284
797
8115
3524
2339
3206
3622
3586
6845
8257
3562
4049
4520
4982
4709
5480
4640
3726
1456
3099
3981
2434
3910
812
2939
4975
5434
2150
2535
3479
3536
6693
4683
5833
2920
1232
4111
6043
3126
7842

In [3]:
status_dict = {
    "note": 1,
    "hadm_id": "Dd",
    "problem": 3,
    "label": 3,
    "cached_messages": None,
}
for field in ["label", "hadm_id", "problem", "cached_messages"]:
    status_dict.pop(field, None)

status_dict

{'note': 1}

In [6]:
lst = [1, 2, 3]
lst2 = [5, 6]
lst.extend(lst2.copy())
lst2.clear()
lst, lst2

([1, 2, 3, 5, 6], [])

### Medical QA

In [ ]:
# 시발 존나 짜증나네. 오지선다 아닌것도 있네 시발. 아오 좆같아시발.
with open("/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_final_with_baseline.json", 'r') as file:
    data = json.load(file)
# for k, v in data[0].items():
    # print(k, v)

import re

for item in data:
    text = item["Question"].split("\n")[-1]
    matches = re.findall(r'\b[A-Z](?=[.:])', text)
    print(matches)


In [8]:
import json
from collections import Counter
import numpy as np

# If you have scikit-learn installed:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    classification_report,
)

def evaluate_multiclass_predictions(
    data,
    aggregator_key="Aggregator",
    aggregator_prediction_key="final_choice",
    baseline_prediction_key="BaselineChoice",
    classes=["A","B","C","D","E"]
):
    
    # ------------------------------------
    # 2. Collect gold labels and predictions
    # ------------------------------------
    gold_labels_agg = []
    pred_labels_agg = []

    gold_labels_base = []
    pred_labels_base = []

    for item in data:
        # The gold label is presumably in item["Answer"], e.g. 'A', 'B', etc.
        gold = item["Answer"]
        
        # Aggregator's prediction
        # e.g. item["Aggregator"]["final_choice"] is 'A', 'B', 'C', etc.
        agg_pred = item[aggregator_key][aggregator_prediction_key]

        # Baseline's prediction
        base_pred = item[baseline_prediction_key]

        gold_labels_agg.append(gold)
        pred_labels_agg.append(agg_pred)

        gold_labels_base.append(gold)
        pred_labels_base.append(base_pred)

    # ------------------------------------------------
    # 3. Build confusion matrices & compute stats
    #    (Using scikit-learn for convenience)
    # ------------------------------------------------

    print("== Multi-Agents ==")
    cm_agg = confusion_matrix(gold_labels_agg, pred_labels_agg, labels=classes)
    print("Confusion Matrix (rows=True label, cols=Predicted):")
    print(cm_agg, "\n")

    accuracy_agg = accuracy_score(gold_labels_agg, pred_labels_agg)
    print(f"Accuracy: {accuracy_agg:.3f}\n")

    # classification_report prints precision, recall, f1 for each class,
    # plus macro/micro/weighted averages
    report_agg = classification_report(
        gold_labels_agg, 
        pred_labels_agg,
        labels=classes,
        zero_division=0  # to handle any class with no predictions
    )
    print(report_agg)

    print("\n== Baseline (ZS COT) ==")
    cm_base = confusion_matrix(gold_labels_base, pred_labels_base, labels=classes)
    print("Confusion Matrix (rows=True label, cols=Predicted):")
    print(cm_base, "\n")

    accuracy_base = accuracy_score(gold_labels_base, pred_labels_base)
    print(f"Accuracy: {accuracy_base:.3f}\n")

    report_base = classification_report(
        gold_labels_base, 
        pred_labels_base,
        labels=classes,
        zero_division=0
    )
    print(report_base)



In [13]:
import re
filtered_data = []
total_data_len = 0
for i in range(1, 4):
    with open("/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step3_final_with_baseline.json", 'r') as file:
        data = json.load(file)
    total_data_len += len(data)

    for item in data:
        text = item["Question"].split("\n")[-1]
        matches = re.findall(r'\b[A-Z](?=[.:])', text)
        if set(matches) == {'A', 'B', 'C', 'D', 'E'}:
            filtered_data.append(item)

evaluate_multiclass_predictions(filtered_data)

== Multi-Agents ==
Confusion Matrix (rows=True label, cols=Predicted):
[[60  0  0  6  0]
 [ 0 33  0  0  0]
 [ 3  0 69  0  0]
 [ 3  6  3 57  0]
 [ 0  0  0  3 75]] 

Accuracy: 0.925

              precision    recall  f1-score   support

           A       0.91      0.91      0.91        66
           B       0.85      1.00      0.92        33
           C       0.96      0.96      0.96        72
           D       0.86      0.83      0.84        69
           E       1.00      0.96      0.98        78

    accuracy                           0.92       318
   macro avg       0.92      0.93      0.92       318
weighted avg       0.93      0.92      0.92       318


== Baseline (ZS COT) ==
Confusion Matrix (rows=True label, cols=Predicted):
[[57  3  3  3  0]
 [ 0 33  0  0  0]
 [ 3  0 66  3  0]
 [ 3  3  3 60  0]
 [ 0  0  0  3 75]] 

Accuracy: 0.915

              precision    recall  f1-score   support

           A       0.90      0.86      0.88        66
           B       0.85      1.00 

In [74]:
len(filtered_data), total_data_len

(267, 357)